In [51]:
import pandas as pd 
import awoc
import numpy as np
import re

In [52]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [53]:
df = pd.read_csv("monumentos_world.csv", index_col=0)

In [54]:
df.head(2)

,Imagen,N.º,PH,Nombre,Ciudad,País,Visitantes/año,Año,#
0,Shwedagon Pagoda 2017.jpg,f/o,—,Gran Pagoda de Shwedagon,Yangon,Bandera de Birmania Birmania,615 763,201747​,—
1,Kerbela Hussein Moschee.jpg,—,—,Tumba del Imán Hussein,Kerbala,Bandera de Irak Irak,45 000 000,?,—


____________

### 1. Empezamos con un EDA básico

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 205
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Imagen          206 non-null    object
 1   N.º             204 non-null    object
 2   PH              205 non-null    object
 3   Nombre          204 non-null    object
 4   Ciudad          159 non-null    object
 5   País            159 non-null    object
 6   Visitantes/año  112 non-null    object
 7   Año             113 non-null    object
 8   #               109 non-null    object
dtypes: object(9)
memory usage: 16.1+ KB


In [56]:
df.shape


(206, 9)

In [57]:
df.columns

Index(['Imagen', 'N.º', 'PH', 'Nombre', 'Ciudad', 'País', 'Visitantes/año',
       'Año', '#'],
      dtype='object')

Cambiamos el nombre de las columnas para que no tenga mayúsculas ni acentos

In [58]:
lista_cols = [x.replace("í", "i").replace("/", "_").lower() for x in df.columns]

In [59]:
# añadimos de manera "manual" a la columna de año "_conteo", para poder diferenciarlo más adelante de la futura columna de "año_monumento", por ejemplo

lista_cols = ['imagen',
 'n.º',
 'ph',
 'nombre',
 'ciudad',
 'pais',
 'visitantes_año',
 'año_conteo',
 '#']

In [60]:
df.columns = lista_cols

In [61]:
df.duplicated().sum()

0

In [62]:
df.isnull().sum() *100 /df.shape[0]

imagen             0.000000
n.º                0.970874
ph                 0.485437
nombre             0.970874
ciudad            22.815534
pais              22.815534
visitantes_año    45.631068
año_conteo        45.145631
#                 47.087379
dtype: float64

In [63]:
df.drop(["ph", "n.º", "#"], axis=1, inplace=True)

_________

### 2. Sacamos la columna de continente

1º "limpiamos" la columna de paises

In [64]:
df["pais"] = df["pais"].str.split()

In [65]:
def pais(col):
    try:
        col = col[-1]
        return col
    except:
        pass

In [66]:
df.head(2)

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo
0,Shwedagon Pagoda 2017.jpg,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​
1,Kerbela Hussein Moschee.jpg,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?


In [67]:
df["pais_corto"] = df["pais"].apply(pais) #Lo almacenamos en una columna nueva por seguir manteniendo la información original

In [68]:
df.head(2)

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto
0,Shwedagon Pagoda 2017.jpg,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,201747​,Birmania
1,Kerbela Hussein Moschee.jpg,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,?,Irak


2º De la columna "limpia de paises" sacamos la columna nueva de continente

In [69]:
my_world = awoc.AWOC()

In [70]:
def continente(col):
    asia = ["Sur", "Birmania" ,"Irak", "Irán", "Japón", 'China', "Turquía",'Israel', 'Yangon','Kerbala', 'Kerala', 'Mashad','Tokio','Benarés','Andhra Pradesh','Prefectura de Chiba','Kamakura','Ise','Beijing','Dazaifu','Saitama','Nueva Delhi','Hangzhou','Taipéi','Nanjing','Changsha','Shanghái','Gansu','Seúl','Taichung','Chongqing','Konya','Xian',"Kevadiya",'Suzhou','Galilea', 'Nazaret','Kaohsiung', 'Jerusalén', 'San Petersburgo', 'Yucatan', 'La Meca','Puerto Rico','Haifa','Taichung', 'Angkor', 'Kevadiya', 'Saitama', 'Agra','China', 'Pekin']
    europa =  ["Unido", 'Francia', 'Portugal', 'Italia', 'Rusia', 'España', 'Polonia', 'Bajos', 'Alemania', 'Austria', 'París','Ourém','Roma','San Giovanni Rotondo','Londres','Lourdes','Venecia','San Petersburgo','Barcelona', 'Madrid','Czestochowa','Florencia', 'Varsovia', 'Valencia','Greenwich','Ámsterdam','Edimburgo','Moscú','Estambul', 'Hohenschwangau','Mariazell','Milán', 'Grecia', 'Alemania', 'Asis','Ciudad del Vaticano', 'Nápoles', 'Varsovia', 'Viena', 'Granada', 'Oswiecim', 'Cordoba', 'Sevilla', 'Amesbury(condad de Wiltshire)', 'Konya','Volgogrado','Pamukkale', 'Kazan','Viena', 'Colonia','Roma', 'Versalles']
    america = ["TRINIDAD AND TOBAGO", "Brasil", "México", "Unidos", 'Aparecida', 'Cuzco', 'Ciudad de México','Washington D. C.','Nueva York','Filadelfia','Los Ángeles','Houston','Quintana Roo','Teotihucán']
    oceania =['Melbourne', 'Sidney']
    africa = ['El Cairo']

    try:
        if col in asia:
            return "Asia"
        elif col in europa:
            return "Europa"
        elif col in america:
            return "America"
        elif col in africa:
            return "Africa"
        else:
            return my_world.get_country_continent_name(col)
    except:
        pass

In [71]:
df["continente"] = df["pais_corto"].apply(continente)

In [72]:
df["continente"].unique()

array(['Asia', None, 'America', 'Europa', 'Oceania'], dtype=object)

_____________

### 3. "Rellenamos" la columna de ciudad con los datos correspondientes

In [73]:
df.ciudad.unique()

array(['Yangon', 'Kerbala', 'Kerala', 'Mashad', 'Tokio', 'Benarés',
       'Andhra Pradesh', nan, '20182\u200b', '?3\u200b', 'Aparecida',
       '20184\u200b5\u200b', 'Ciudad de México', 'París',
       'Prefectura de Chiba', 'Kamakura', '20121\u200b', '201921\u200b',
       'Ourém', 'Ise', '20188\u200b', 'Washington D. C.', '201810\u200b',
       'Beijing', '201611\u200b', '2018 12\u200b', '201713\u200b', 'Roma',
       'Flag of the United Arab Emirates.svg Emiratos Árabes Unidos',
       'Dazaifu', 'San Giovanni Rotondo', 'Nueva York', 'Londres',
       'Lourdes', '201815\u200b', '201616\u200b', 'Venecia', 'Saitama',
       'San Petersburgo', 'Filadelfia', 'Barcelona', 'Nueva Delhi',
       '?7\u200b', 'Madrid', 'Czestochowa', 'Florencia', 'Hangzhou',
       'Washington, D.C.', '201820\u200b', 'Taipéi', '201922\u200b',
       'Nanjing', '2018 23\u200b', '2018', 'Changsha', 'Shanghái',
       'Gansu', 'Seúl', '[[]]', 'Varsovia', '201725\u200b',
       '201726\u200b', 'Taichung', '2018

In [74]:
df[df["ciudad"]== '20182\u200b']

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
8,( Patrimonio de la Humanidad (parte de «Palaci...,17 000 000+,20182​,[—],NaN,NaN,—,None


In [75]:
df.iloc[8,2] = "Pekin"

In [76]:
df.iloc[8,2]

'Pekin'

_______________________

### 4. Limpiamos la columna de año

In [77]:
df.año_conteo.unique() # Vemos que el año cuando se tomaron el conteo de gente son los primeros 4 numeros, como no sabemos a qué hace referencia los siguientes los quitaremos.

array(['201747\u200b', '?', '20121\u200b', nan, '20176\u200b', '?7\u200b',
       '2018 9\u200b', '201921\u200b', '—', '201949\u200b', '20189\u200b',
       '201817\u200b', '2018 18\u200b', '201850\u200b', '201919\u200b',
       '201838\u200b', '2019 51\u200b', '201952\u200b', '201827\u200b',
       '201753\u200b', '201624\u200b', '201854\u200b', '201855\u200b',
       '201831\u200b', '201832\u200b', '201933\u200b', '201856\u200b',
       '2017 57\u200b', '201858\u200b', '201859\u200b', '201836\u200b',
       '201760\u200b', '201837\u200b', '201739\u200b', '201761\u200b',
       '201842\u200b', '2017', '201843\u200b', '201744\u200b'],
      dtype=object)

In [78]:
def limpia_año(col):
    patron = r"\d{4}"

    try:
        return re.findall(patron, col)[0]
    except:
        pass

In [79]:
df["año_conteo"] = df["año_conteo"].apply(limpia_año)

In [80]:
df.head(2)

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Shwedagon Pagoda 2017.jpg,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,2017,Birmania,Asia
1,Kerbela Hussein Moschee.jpg,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,None,Irak,Asia


In [81]:
df.año_conteo.unique()

array(['2017', None, '2012', '2018', '2019', '2016'], dtype=object)

Convertimos a float y no a int porque la columna tiene NaNs

In [82]:
df["año_conteo"] = df["año_conteo"].astype(float)

____________________

### 5. "Limpiamos" y cambiamos el tipo de dato de la columna "visitantes_año"

In [83]:
df["visitantes_año"].unique()

array(['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
       '18 250 000', nan, '13 000 000', '10 600 000', '10 500 000',
       '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
       '7 000 000', '2019 14\u200b', '6 600 000', '6 479 548',
       '6 239 983', '6 098 340', '6 011 007', '6 000 000', '5 500 000',
       '5 300 000', '5 000 000', '4 956 529', '4 719 148', '4 652 865',
       '4 576 436', '4 500 000', '4 425 699', '4 400 000', '4 391 861',
       '4 335 431', '4 200 000', '4 074 403', '3 932 938', '3 832 373',
       '3 670 000', '3 600 000', '3 540 000', '3 500 000', '3 354 161',
       '3 286 000', '3 273 867', '3 232 730', '3 200 000', '3 143 626',
       '3 100 000', '3 098 985', '2 963 857', '2 893 000', '2 830 000',
       '2 817 386', '2 800 000', '2 787 770', '2 780 000', '2 774 103',
       '2 717 565', '2 661 615', '2 637 567', '2 600 000+', '2 596 725',
       '2 565 474', '2 560 000', '2 550 000', '2 549 833', '2 520 000',
 

1º Eliminamos los valores corruptos

In [84]:
lista = [ "2019 14\u200b", "201935\u200b", "201616\u200b", "+" ] #, " "]

for x in lista:
    df["visitantes_año"] = df["visitantes_año"].str.replace(x, "")

/tmp/ipykernel_538/4176642583.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["visitantes_año"] = df["visitantes_año"].str.replace(x, "")


In [85]:
df["visitantes_año"].unique()

array(['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
       '18 250 000', nan, '13 000 000', '10 600 000', '10 500 000',
       '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
       '7 000 000', '', '6 600 000', '6 479 548', '6 239 983',
       '6 098 340', '6 011 007', '6 000 000', '5 500 000', '5 300 000',
       '5 000 000', '4 956 529', '4 719 148', '4 652 865', '4 576 436',
       '4 500 000', '4 425 699', '4 400 000', '4 391 861', '4 335 431',
       '4 200 000', '4 074 403', '3 932 938', '3 832 373', '3 670 000',
       '3 600 000', '3 540 000', '3 500 000', '3 354 161', '3 286 000',
       '3 273 867', '3 232 730', '3 200 000', '3 143 626', '3 100 000',
       '3 098 985', '2 963 857', '2 893 000', '2 830 000', '2 817 386',
       '2 800 000', '2 787 770', '2 780 000', '2 774 103', '2 717 565',
       '2 661 615', '2 637 567', '2 600 000', '2 596 725', '2 565 474',
       '2 560 000', '2 550 000', '2 549 833', '2 520 000', '2 460 000',
  

2º convertimos a número

In [86]:
#df["visitantes_año"] = pd.to_numeric(df["visitantes_año"])   
# Dejo comentada la fila porque si convierto a float luego no puedo filtrar por value_counts

In [87]:
df.head(2)

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Shwedagon Pagoda 2017.jpg,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,2017.0,Birmania,Asia
1,Kerbela Hussein Moschee.jpg,Tumba del Imán Hussein,Kerbala,"[Bandera, de, Irak, Irak]",45 000 000,NaN,Irak,Asia


______

### 6. Años anteriores a 2018

In [88]:
df["año_conteo"].unique()

array([2017.,   nan, 2012., 2018., 2019., 2016.])

In [89]:
lista_años = [2012 ,2016 , 2017]

df_anteriores = df[df["año_conteo"].isin(lista_años) ]

In [90]:
df_anteriores

,imagen,nombre,ciudad,pais,visitantes_año,año_conteo,pais_corto,continente
0,Shwedagon Pagoda 2017.jpg,Gran Pagoda de Shwedagon,Yangon,"[Bandera, de, Birmania, Birmania]",615 763,2017.0,Birmania,Asia
4,Meiji shrine from right.jpg,Santuario Meiji,Tokio,"[Bandera, de, Japón, Japón]",30 000 000,2012.0,Japón,Asia
5,—,Templo de Kashi Vishwanath,Benarés,"[Bandera, de, la, India, India]",21 900 000,2012.0,India,Asia
6,—,Templo de Tirupati Tirumala Devasthanams,Andhra Pradesh,"[Bandera, de, la, India, India]",18 250 000,2012.0,India,Asia
11,Basilica of the National Shrine of Our Lady of...,Basílica de Nuestra Señora Aparecida,Aparecida,"[Bandera, de, Brasil, Brasil]",13 000 000,2017.0,Brasil,America
16,Naritasan-Shinshoji-Temple.Great-Main-Hall.jpg,Templo Naritasan Shinshoji,Prefectura de Chiba,"[Bandera, de, Japón, Japón]",10 000 000,2012.0,Japón,Asia
17,TsurugaokaHachiman4269.jpg,Tsurugaoka Hachiman-gū,Kamakura,"[Bandera, de, Japón, Japón]",10 000 000,2012.0,Japón,Asia
25,IseShrine.jpg,Santuario de Ise,Ise,"[Bandera, de, Japón, Japón]",8 500 000,2012.0,Japón,Asia
44,20100719 Dazaifu Tenmangu Shrine 3328.jpg,Dazaifu Tenman-gū,Dazaifu,"[Bandera, de, Japón, Japón]",6 600 000,2012.0,Japón,Asia
45,Chiesa San Pio da Pietrelcina.JPG,Santuario del Padre Pío en San Giovanni Rotondo,San Giovanni Rotondo,"[Flag, of, Italy.svg, Italia]",6 600 000,2012.0,Italia,Europa


In [91]:
df_anteriores.shape

(31, 8)

31 valores a actualizar en la columna de "visitantes_año"

__________

### 6.2 años posteriores a 2018

In [92]:
lista_años_2 = [2018, 2019]

df_posteriores = df[df["año_conteo"].isin(lista_años_2) ]

In [98]:
df_posteriores.shape

(69, 8)

________________

### 6.3 Sacar valores nulos

Tenemos 106 valores nulos en la columna "visitantes_año". porque en el filtro de anteriores y posteriores tenemos un total de 100 y el total de las filas es de 206. Lo dejo como pendientes de sacar.

_______________

### Otros filtros

In [93]:
df_asia = df[df["continente"]== "Asia"]
df_america = df[df["continente"]== "America"]
df_europa = df[df["continente"]== "Europa"]
df_africa = df[df["continente"]== "Africa"]
df_oceania = df[df["continente"]== "Oceania"]

In [94]:
# Posible "filtro" para quitar valores nans



# lista_visitantes = ['615 763', '45 000 000', '40 000 000', '30 000 000', '21 900 000',
#        '18 250 000', '13 000 000', '10 600 000', '10 500 000',
#        '10 000 000', '9 400 000', '8 500 000', '7 804 683', '7 390 000',
#        '7 000 000',  '6 600 000', '6 479 548',
#        '6 239 983', '6 098 340', '6 011 007', '6 000 000', '5 500 000',
#        '5 300 000', '5 000 000', '4 956 529', '4 719 148', '4 652 865',
#        '4 576 436', '4 500 000', '4 425 699', '4 400 000', '4 391 861',
#        '4 335 431', '4 200 000', '4 074 403', '3 932 938', '3 832 373',
#        '3 670 000', '3 600 000', '3 540 000', '3 500 000', '3 354 161',
#        '3 286 000', '3 273 867', '3 232 730', '3 200 000', '3 143 626',
#        '3 100 000', '3 098 985', '2 963 857', '2 893 000', '2 830 000',
#        '2 817 386', '2 800 000', '2 787 770', '2 780 000', '2 774 103',
#        '2 717 565', '2 661 615', '2 637 567', '2 600 000+', '2 596 725',
#        '2 565 474', '2 560 000', '2 550 000', '2 549 833', '2 520 000',
#         '2 460 000', '2 431 073', '2 400 000',
#         '2 340 000', '2 313 000', '2 304 404', '2 300 000',
#        '2 250 000', '2 240 000', '2 231 000', '2 227 773', '2 192 200',
#        '2 190 000', '2 165 601', '2 161 160', '2 148 538', '2 111 730',
#        '2 111 578', '2 100 000', '2 054 719', '2 000 000', '1 981 000',
#        '1 700 000', '1 657 446', '1 583 260', '1 552 481', '1 500 000',
#        '1 300 000', '1 000 000+', '1 000 000']
# 
# df_2 = df[df["visitantes_año"].isin(lista_visitantes)]

In [95]:
# Esta celda y la siguiente para filtrar por los top 10 de visitas según contiente. No aplicar hasta decidir como rellenar los datos valores no actualizados o nans de la columna "visitantes_año"

# df_europa["visitantes_año"].value_counts()[:10]

In [96]:
# lista_europa = [ "10 500 000" ,"2 161 160","2 637 567" ,"2 549 833" ,"2 300 000" ,"2 231 000" ,"2 227 773" ,"2 192 200" ,"2 165 601" ,"2 148 538"  ]
# df_europa_2 = df_europa[df_europa["visitantes_año"].isin(lista_europa)]
# df_europa_2



# df_total = pd.concat([df_europa_2,df_america_2,df_asia_2,df_oceania, df_africa], axis=0).reset_index()